<a href="https://colab.research.google.com/github/theaidran/ollama_youtube_summarize/blob/main/ollama_youtube_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title #####  Put a YouTube link below and hit Run a cell. Use any-language video and get any-language summary. Quick access to this colab notebook. [tinyurl.com/summarisation](https://tinyurl.com/summarisation) { display-mode: "form" }
%config InteractiveShell.ast_node_interactivity="all" #update python display config
!pip install openai==1.55.3 > /dev/null
!pip install httpx==0.27.2 > /dev/null


In [3]:
# @title  Summarize { run: "auto", display-mode: "form" }
# All credits to https://github.com/martinopiaggi/summarize
# Good to see links:
# https://github.com/hsiehjackson/RULER
# https://ollama.com/library
# https://amgadhasan.substack.com/p/sota-asr-tooling-long-form-transcription


#1 Choose source url
Link = "https://www.youtube.com/watch?v=t6VoqbAh9HM" #@param {type:"string"}
Type_of_source = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive video link','Dropbox video link', 'Local file']
Type = Type_of_source
URL = Link
Translate_to = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}

#@markdown ---
#markdown Insert your API key depending on which endpoint you want to use
# API Configuration
api_key = "ollama"
api_endpoint = "Custom"
ollama_model = "llama3" # param ['llama3', 'phi3', 'vicuna:13b-v1.5-16k-q4_0']

endpoints = {
    "Groq": "https://api.groq.com/openai/v1",
    "OpenAI": "https://api.openai.com/v1",
    "Custom": "http://localhost:11434/v1"  # ollama endpoint
}
base_url = endpoints.get(api_endpoint)

models = {
    "Groq": "llama3-8b-8192",
    "OpenAI": "gpt-3.5-turbo",
    "Custom": ollama_model  # Placeholder for any custlom model
}
model = models.get(api_endpoint)

use_Youtube_captions = True #@param {type:"boolean"}
# @markdown  * otherwise transcribe audio


#2 launch ollama server with llama3
try: # to faster next start
  import pytubefix
except ImportError:
  !curl -fsSL https://ollama.com/install.sh | sh
  !tmux new -d ollama serve
  !ollama run {ollama_model} !


#3 install libs: youtube and whisperx
import subprocess
import re
import os
import IPython
from IPython.display import clear_output

def install_whisperx():
    try:
      import whisperx
    except ImportError:
      print("installing whisperx...may take a few minutes")
      !pip install torch==2.5.1+cu121 torchaudio==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121
      !pip install git+https://github.com/m-bain/whisperx.git > /dev/null
      !apt-get update
      !apt-get install libcudnn8=8.9.2.26-1+cuda12.1
      !apt-get install libcudnn8-dev=8.9.2.26-1+cuda12.1
      !python -c "import torch; torch.backends.cuda.matmul.allow_tf32 = True; torch.backends.cudnn.allow_tf32 = True"
      !ln -s /usr/lib/x86_64-linux-gnu/libcudnn_ops_infer.so /usr/lib/x86_64-linux-gnu/libcudnn_ops_infer.so.8
      clear_output()

if use_Youtube_captions:
  !pip install youtube-transcript-api
  from youtube_transcript_api import YouTubeTranscriptApi

if (not Type == "Youtube video or playlist") or (not use_Youtube_captions):
  install_whisperx()

try:
  import deep_translator
except ImportError:
  !pip install -U deep-translator

try:
  import openai
except ImportError:
  !pip install openai==1.55.3
  !pip install httpx==0.27.2  #clear_output()

import openai
client = openai.OpenAI(api_key=api_key, base_url=base_url)

if Type == "Youtube video or playlist":
  try:
    import pytubefix
  except ImportError:
    #!pip install git+https://github.com/pytube/pytube
    !pip install git+https://github.com/JuanBindez/pytubefix
    #clear_output()
    from pytubefix import YouTube

if Type == "Google Drive video link":
  from google.colab import drive
  drive.mount('/content/drive')

#4 Video fetching
skip_transcription=False
text = ""
textTimestamps = ""

def seconds_to_time_format(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

def download_youtube_audio_only(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only()
    saved_path = audio_stream.download(output_path=".", skip_existing=True)
    return saved_path

def check_if_captions_exist(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)

    try:
      transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
      return (True)
    except:
      return (False)

def download_youtube_captions(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id) #languages=['en']
    except:
      for available_transcript in transcript_list:
        if available_transcript.is_translatable:
          transcript = available_transcript.translate('en').fetch()
          break

    text = ""
    for entry in transcript:
            start_time = seconds_to_time_format(entry['start'])
            text += f"{start_time} {entry['text'].strip()}\n"

    transcript_file_name = f"{video_id}_captions.md"

    with open(transcript_file_name, 'w', encoding='utf-8') as f:
      f.write(text)

    return text,transcript_file_name

if Type == "Youtube video or playlist":
    #clean youtube url from timestamp
    URL = re.sub('\&t=\d+s?', '', URL)

    if use_Youtube_captions and check_if_captions_exist(URL):
      text, transcript_file_name = download_youtube_captions(URL)
      skip_transcription=True
    else:
      install_whisperx()
      video_path_local =  download_youtube_audio_only(URL)

elif Type == "Google Drive video link":
  subprocess.run(['ffmpeg', '-y', '-i', 'drive/MyDrive/' + URL, '-vn', '-acodec', 'pcm_s16le',
                  '-ar', '16000', '-ac', '1', 'gdrive_audio.wav'], check=True)
  video_path_local = "gdrive_audio.wav"

elif Type == "Dropbox video link":
    subprocess.run(['wget', URL, '-O', 'dropbox_video.mp4'], check=True)
    subprocess.run(['ffmpeg', '-y', '-i', 'dropbox_video.mp4', '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'dropbox_video_audio.wav'], check=True)
    video_path_local = "dropbox_video_audio.wav"

elif Type == "Local file":
    local_file_path = Source
    subprocess.run(['ffmpeg', '-y', '-i', local_file_path, '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'local_file_audio.wav'], check=True)
    video_path_local = "local_file_audio.wav"


#5 transcription using whisperx
if not skip_transcription:
  import whisperx
  language = "auto" # param {type:"string"}
  #initial_prompt = "" # @param {type:"string"}

  asr_options={"temperatures": 0, "beam_size": 1, "without_timestamps": True,}
  # Run on GPU with FP16
  modelWhisperX = whisperx.load_model("large-v2", device="cuda", compute_type="float16", asr_options=asr_options)

  transcription = modelWhisperX.transcribe(str(video_path_local), batch_size=16,
                                    language=None if language == "auto" else language,
                                    task="translate")

  transcript_file_name = os.path.splitext(video_path_local)[0]+'.md'

  with open(transcript_file_name, 'w') as f:
    for segment in transcription["segments"]:
        start_time = seconds_to_time_format(segment['start'])
        text += f"{start_time} {segment['text'].strip()} "

    f.write(text)


#6 Make a summary
prompt_type = "Summarization"  # param ['Summarization', 'Only grammar correction with highlights']
# markdown Set the number of parallel API calls (be mindful of usage rate limits)
parallel_api_calls = 1 # param


# Define your prompts using a dictionary for easier management
prompts = {
    'Summarization': """Summarize the video transcript excerpt, including a bold title. Use narration in third-person. Write the summary as if you are continuing a conversation without needing to signal a beginning. Here is the transcript: """,
    'Only grammar correction with highlights': """Repeat the following text correcting any grammatical errors and formatting error. Highlight only the important quote (if there are any) with **markdown bold notation**. Focus solely on the essence of the content as if you are continuing a conversation without using any form of introduction like 'Here's the corrected text:'. Here is the text to fix: """
}

# Select the appropriate prompt
summary_prompt = prompts[prompt_type]



def extract_and_clean_timestamps(text_chunks):
    timestamp_pattern = re.compile(r'(\d{2}:\d{2}:\d{2})')
    cleaned_texts = []
    timestamp_ranges = []
    for chunk in text_chunks:
        timestamps = timestamp_pattern.findall(chunk)
        if timestamps:
            for timestamp in timestamps:
                # Remove each found timestamp from the chunk
                chunk = chunk.replace(timestamp, "")
            timestamp_ranges.append(timestamps[0])  # Assuming you want the first timestamp per chunk
        else:
            timestamp_ranges.append("")
        cleaned_texts.append(chunk.strip())  # Strip to remove any leading/trailing whitespace
    return cleaned_texts, timestamp_ranges

def format_timestamp_link(timestamp):
    if Type == "Youtube video or playlist":
      hours, minutes, seconds = map(int, timestamp.split(':'))
      total_seconds = hours * 3600 + minutes * 60 + seconds
      return f"{timestamp} - {URL}&t={total_seconds}"
    else:
      return f"{timestamp}"

import concurrent.futures
import time
from deep_translator import GoogleTranslator

def summarize(prompt):
    completion = client.chat.completions.create(
            model=model,
            messages=[
            {"role": "system", "content": ""},
            {"role": "user", "content": summary_prompt + " " + prompt}
            ],
            max_tokens=4096
    )
    return completion.choices[0].message.content

def process_and_summarize(text,chunk_size):
    overlap_size = 20
    texts = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap_size)]
    cleaned_texts, timestamp_ranges = extract_and_clean_timestamps(texts)
    summaries = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=parallel_api_calls) as executor:
        future_to_chunk = {executor.submit(summarize, text_chunk): idx for idx, text_chunk in enumerate(cleaned_texts)}
        for future in concurrent.futures.as_completed(future_to_chunk):
            idx = future_to_chunk[future]
            try:
                summarized_chunk = future.result()
                summary_piece = format_timestamp_link(timestamp_ranges[idx]) + " " + summarized_chunk
                summary_piece += "\n"
                summaries.append((idx, summary_piece))
            except Exception as exc:
                print(f'Chunk {idx} generated an exception: {exc}')
                # Resubmit the task with the new model
                time.sleep(10)
                future_to_chunk[executor.submit(summarize, texts[idx])] = idx

    summaries.sort()  # Ensure summaries are in the correct order
    final_summary =""
    if Translate_to != "English":
      translated_summaries = []
      for _,summary in summaries:
        translated_summary = GoogleTranslator(source='auto', target=language_codes[Translate_to]).translate(summary)
        translated_summaries.append(translated_summary)
        final_summary += translated_summary + "\n\n"
      final_summary = final_summary.strip()
    else :
      final_summary = "\n\n".join([summary for _, summary in summaries])

    # Save the final summary
    final_name = transcript_file_name.replace(".md", "_FINAL_"+str(chunk_size)+".md") if Type != "Dropbox video link" else "final_dropbox_video.md"
    with open(final_name, 'w') as f:
        f.write(final_summary)
    return (final_summary)

from IPython.core.interactiveshell import InteractiveShell # Unblock output
InteractiveShell.ast_node_interactivity = "all"            # It starts to work after restart cell
#%config InteractiveShell.ast_node_interactivity="all"
clear_output()
print ("Generating summary...")
general_summary = process_and_summarize(text,15000) # max string size = max_tokens x 4
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary)
final_summary = process_and_summarize(text,3072)
clear_output()
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary + "\n\n" + "\n\n **Long Summary:** \n\n" + final_summary )





 **Short Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **Creating a Drone Review Website with AI**

In this video transcript excerpt, a narrator walks through the process of creating a website reviewing drones using artificial intelligence (AI) tools. The narrator starts by setting up the site's content sections using Fontjoy to create pairings for font styles and Well to generate content. They then use CLAude to write a heading and subheading in the style of Alex Horoi.

The narrator discusses the importance of well-written copy and uses Well again to write additional headings and subheadings. They also use AI image generation tools, such as Midjourney, Dolly 3, or Canva, to generate images for the website's featured posts.

Throughout the process, the narrator emphasizes the importance of finding the right AI tools for specific tasks, such as generating content, writing copy, and creating images. They demonstrate how to use these tools to create a functional website without needing extensive coding knowledge.


00:11:15 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=675 **Automating Website Creation with AI**

The speaker, Matt Wolf, is showing off some exciting AI tools that can help automate website creation. He starts by generating a thumbnail using Canva's Magic Media feature, opting for a filmic style with a camera drone flying above a city skyline. After tweaking the image, he uses AI to remove the background and adds text with a font similar to what's used on his website.

Next, Matt demonstrates how Canva can be used to create featured images for blog posts. He selects one of his own uploaded images, removes the background using Canva's AI feature, and overlaps it with the drone image. Finally, he adds text with the title "Best Drones for City Flights."

**The Power of AI in Website Creation**

Matt emphasizes that AI is not yet ready to create a beautiful website without some human effort. However, it can help automate many tasks, such as generating images and writing copy. He highlights several AI tools used throughout his process, including:

1. Canva's Magic Media feature for image generation
2. Fontjoy for font pairing suggestions
3. ChatGPT or Claude for copywriting assistance
4. Image generators like Midjourney, Leonardo, Dolly 3, and Bing co-pilot

**Conclusion**

Matt concludes by thanking the audience and encouraging viewers to share the video with anyone trying to build a website but struggling to get started. He also invites subscribers to join his free newsletter at Future Tools, where they can learn about new AI tools and gadgets, including being entered to win an Insta 360 X4 camera each month.




 **Long Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **AI-Powered Website Building: Exploring Reloom**

As we dive into the world of AI-enabled tools, [the speaker] emphasizes the importance of having a website for everyone. To make this process easier, they introduce Reloom, a tool that maps out an entire website using artificial intelligence. With a free plan available forever, it's a great starting point for creating a simple website without requiring technical knowledge.

Within Reloom, [the speaker] creates a new project and defines the prompt as a website reviewing and comparing camera drones. The AI-generated site map includes essential pages such as the homepage, hero section, feature list, reviews, and more. Each page can be further customized by generating content and designing the layout using Reloom's intuitive interface.

By clicking on "generate" sections like "about us," [the speaker] has already built out a complete site map with all necessary pages. From there, they can explore each page by scrolling up to the top and clicking on the "wireframe" button. This feature provides a rough design for each page, allowing [the speaker] to visualize the final product.


00:02:18 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=138 **Building a Website in Minutes with AI**

As we continue, Wolf introduces us to the power of AI in building websites. He recommends Hostinger, the sponsor of the video, for hosting services. By using their "Business website builder," you can get started with AI-powered tools on WordPress for as little as $2.99 a month, with two months free. By selecting the Business plan and adding it to your cart, you'll have access to most of the AI tools.

Wolf shows us how to fill out the details, apply the coupon code "Matt Wolf" (no spaces) to get an even bigger discount, and then create a new website with Hostinger's website builder. He chooses to let AI create his website in minutes, calling it "Drone World," a blog dedicated to comparing and reviewing camera drones.

With just a few clicks, the AI does the work of setting up everything for us. We can already see the website live, with pictures automatically pulled in from drone photography. Wolf wants to edit the site and play with the color scheme using their third tool, ColorM.


00:04:34 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=274 **Unlocking Color Palettes with Machine Learning**

As you continue, the narrator jumps back into their conversation, saying "now that we've got our color scheme dialed in" - referring to the AI-generated color palette they had previously explored on Colormind. The user selected a few colors and locked them in, allowing the tool to generate variations that matched well with those initial choices. With this new color palette, the narrator suggests it's ready for use on their website, simply swapping out the existing color codes for the new ones.

Moving along, the next topic is font selection, where Font Joy comes into play. This AI-powered site uses machine learning to find harmonious font combinations. The user can generate different font styles until they find one they like, in this case, Robotto condensed. With that main font chosen, the narrator locks it in and generates additional font options that pair well with it. After exploring options, they settle on Work Sands and Hind Vad, which they select for their website's headers and subheaders respectively.


00:06:48 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=408 **Building a Website with AI-powered Content and Design**

As we continue to build our website, the host has already been set up on Hostinger. We've also nailed down the color scheme using Color Mind, and chosen a font pairing through Font Joy.com. Now, it's time to start filling in some content. To do this, the creator is considering using either Chat GPT or Claude AI models to generate copy. The main challenge is crafting a prompt that gets the desired writing style.

To test this, the creator has started a prompt for Claude, outlining details about the site - a website that reviews drones to help consumers make informed decisions. They've also provided specific instructions, asking for some website copies that will entice users when they land on the site. To take it up a notch, the creator has even given credit to inspiration from notable marketers, Alex Horoi and David Ogie.

After providing these details, Claude has generated a heading and subheading in the style of Alex Horoi and David Ogoli. The result is: "The Ultimate Drone Buyer Guide: New Heights with the Perfect UAV" as the heading, and "Discover the best drones for your needs and budget: Expert reviews, side-by-side comparisons, and insider tips to help you make an informed decision."


00:09:01 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=541 **Website Optimization Strategies**

As we continue, our speaker is sharing their approach to optimizing website sections. They begin by copying headings and subheadings from the original content and replacing them on the new page. This step is repeated for each section of the website. Next, they discuss generating images using various tools such as DJI Mavic 2 and other camera drones. They also mention Mid Journey, a tool that can be used to generate high-quality images for free, and Dolly 3, which offers similar features.

Once the images are generated, our speaker uses them to replace existing ones on the website, making sure to center them properly using the image grid feature. They also highlight Canva as their go-to tool for creating featured images for blog posts, praising its ease of use in adding text and removing backgrounds from images.

The conversation continues with a focus on optimizing website content, exploring various tools and methods for effective image generation and placement.


00:11:27 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=687 **Customizing Featured Images in Canva**

As we continue exploring the features of Canva, [they] decide to use Magic media's built-in image generator to create consistent styles for their blog post featured images. They start by generating images with different styles and tone, eventually settling on a camera drone flying above a city skyline. With this newly created image, they want it to stand out like a YouTube thumbnail, so [they] set about customizing it.

To make the image more engaging, [they] choose an image of themselves from their own uploads and use AI background remover to eliminate the extraneous background. This paid Canva feature works well in removing the background, and [they] proceed to pull off a fun pose with the drone as the focal point. To make it pop even more, [they] add a white outline around themselves using the Shadows feature.

Finally, [they] decide to overlay some text on the image, selecting the font style "Robotto Condensed" to match their website's existing typography. With the text added and stretched to cover the entire image, the featured image is now ready to be shared with their audience.


00:13:31 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=811 **Streamlining Website Creation with AI**

As the narration continues, the speaker walks us through a process of creating a website using various AI-enhanced tools. They start by rapidly designing a featured image using Canva, adding effects like a light drop shadow to make it pop off the screen. The featured image is then downloaded as a JPEG and used on their website. The speaker highlights that they've already created the entire website using AI with just a few clicks on Hostinger, using coupon code Matt wolf for an extra discount.

The walkthrough continues, explaining how AI tools like Color Mind helped figure out a color scheme, Font Joy provided font pairings, and either ChatGPT or Claude wrote the copy for the website. For image generation, they used Mid Journey, but other options like Leonardo, Dolly 3, and Bing Co-pilot are also possible.

The final step involves using Canva to create featured images for blog posts on the website, showcasing the entire process of building a decent website with AI tools. While admitting that current AI capabilities may not produce a fully polished, multi-million-dollar-like website, the speaker emphasizes that with some extra effort and tweaking, you can get very close to achieving that look. The video concludes by highlighting the featured AI tools demonstrated in the walkthrough, inviting viewers to participate in future videos exploring more AI-powered website creation techniques.


00:15:55 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=955 **Free AI Tools and Giveaways with Future Tools**

The host of the YouTube channel continues to promote Future Tools, a curated list of cool AI tools that they update almost daily. They invite viewers to sign up for their free newsletter, which will send them the latest news and coolest tools. In addition, as a subscriber, you'll be entered to win exciting giveaways every month, such as an Insta 360 X4 camera, just by being subscribed to the channel and joining the newsletter. The host encourages viewers to join both the YouTube channel and the free newsletter to stay in the loop with AI news and tools.
